In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Reading the dataset file:
df = pd.read_csv('cleaned_food_drive_data-2.csv')
# Setting the option to display 100 rows:
pd.set_option('display.max_rows', 100)

In [3]:
df.shape

(450, 12)

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           450 non-null    object 
 1   Location                       450 non-null    object 
 2   Stake                          450 non-null    object 
 3   # of Adult Volunteers          450 non-null    int64  
 4   # of Youth Volunteers          450 non-null    int64  
 5   Donation Bags Collected        450 non-null    int64  
 6   Time to Complete (min)         450 non-null    float64
 7   Completed More Than One Route  450 non-null    int64  
 8   Ward/Branch                    450 non-null    object 
 9   Routes Completed               450 non-null    int64  
 10  Doors in Route                 450 non-null    int64  
 11  Time Spent                     450 non-null    float64
dtypes: float64(2), int64(6), object(4)
memory usage: 4

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

,0
Date,0
Location,0
Stake,0
# of Adult Volunteers,0
# of Youth Volunteers,0
Donation Bags Collected,0
Time to Complete (min),0
Completed More Than One Route,0
Ward/Branch,0
Routes Completed,0


In [7]:
df['Stake'].unique()

array(['Bonnie Doon Stake', 'Gateway Stake', 'Edmonton North Stake',
       'Riverbend Stake', 'YSA Stake'], dtype=object)

In [8]:
def generate_route_names(df):
    """Generates new route names based on stakes."""
    route_counters = {}  # Store counters for each stake
    new_route_names = []

    for index, row in df.iterrows():
        stake = row['Stake']
        if stake not in route_counters:
            route_counters[stake] = 1  # Initialize counter for new stake
        else:
            route_counters[stake] += 1  # Increment counter for existing stake

        new_route_names.append(f"{stake} {route_counters[stake]}")  # Create new route name

    df['New Route Number/Name'] = new_route_names  # Add new column to DataFrame
    return df

# Apply the function to generate new route names
df = generate_route_names(df)

# Print the updated DataFrame (optional)
print(df)

                    Date                             Location  \
0    2023-09-23 10:36:10                   Londonderry Chapel   
1    2023-09-23 10:36:52                 Gateway Stake Centre   
2    2023-09-23 10:40:22                 Gateway Stake Centre   
3    2023-09-23 10:41:08                 Gateway Stake Centre   
4    2023-09-23 10:44:10                   Londonderry Chapel   
..                   ...                                  ...   
445  2023-09-27 14:24:33                   North Stake Centre   
446  2023-09-27 14:24:35                   North Stake Centre   
447  2023-09-28 20:44:23  Parkland (Spruce Grove/Stony Plain)   
448  2023-10-15 11:15:43                   North Stake Centre   
449  2024-09-10 16:03:36                   Londonderry Chapel   

                    Stake  # of Adult Volunteers  # of Youth Volunteers  \
0       Bonnie Doon Stake                      1                      3   
1           Gateway Stake                      3                     

In [9]:
# Group by 'Stake' and count unique routes
total_routes_by_stake = df.groupby('Stake')['New Route Number/Name'].nunique()

# Create a new column 'TotalRoutes' and assign the route counts using map
df['TotalRoutes'] = df['Stake'].map(total_routes_by_stake)

# Display the updated DataFrame (optional) to see the new column
print(df[['Stake', 'TotalRoutes']].head())  # Display the first few rows with stake and total routes

               Stake  TotalRoutes
0  Bonnie Doon Stake          127
1      Gateway Stake          192
2      Gateway Stake          192
3      Gateway Stake          192
4  Bonnie Doon Stake          127


In [10]:
df

,Date,Location,Stake,# of Adult Volunteers,# of Youth Volunteers,Donation Bags Collected,Time to Complete (min),Completed More Than One Route,Ward/Branch,Routes Completed,Doors in Route,Time Spent,New Route Number/Name,TotalRoutes
0,2023-09-23 10:36:10,Londonderry Chapel,Bonnie Doon Stake,1,3,14,25.0,0,Clareview Ward,2,14,25.000000,Bonnie Doon Stake 1,127
1,2023-09-23 10:36:52,Gateway Stake Centre,Gateway Stake,3,1,18,25.0,1,Crawford Plains Ward,2,144,30.000000,Gateway Stake 1,192
2,2023-09-23 10:40:22,Gateway Stake Centre,Gateway Stake,2,0,20,30.0,0,Silver Berry Ward,2,186,30.000000,Gateway Stake 2,192
3,2023-09-23 10:41:08,Gateway Stake Centre,Gateway Stake,2,0,20,25.0,1,Crawford Plains Ward,1,196,30.000000,Gateway Stake 3,192
4,2023-09-23 10:44:10,Londonderry Chapel,Bonnie Doon Stake,1,0,2,5.0,0,Londonderry Ward,2,1,-30.894484,Bonnie Doon Stake 2,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2023-09-27 14:24:33,North Stake Centre,Edmonton North Stake,2,0,20,20.0,0,Namao Ward,2,150,45.000000,Edmonton North Stake 53,57
446,2023-09-27 14:24:35,North Stake Centre,Edmonton North Stake,2,0,20,20.0,0,Namao Ward,2,150,45.000000,Edmonton North Stake 54,57
447,2023-09-28 20:44:23,Parkland (Spruce Grove/Stony Plain),Edmonton North Stake,2,4,51,60.0,1,Stony Plain Ward,2,196,120.000000,Edmonton North Stake 55,57
448,2023-10-15 11:15:43,North Stake Centre,Edmonton North Stake,3,0,78,150.0,1,Griesbach Ward,3,600,150.000000,Edmonton North Stake 56,57


In [11]:
# Group by 'Route Number/Name' and calculate the average '# of Doors in Route' for each group
average_doors_per_route = df.groupby('New Route Number/Name')['Doors in Route'].mean()

# Create a new column 'AvgDoorsPerRoute' and assign the average doors for each route using map
df['AvgDoorsPerRoute'] = df['New Route Number/Name'].map(average_doors_per_route)

# Print the updated DataFrame to see the new column
print(df[['New Route Number/Name', 'AvgDoorsPerRoute']].head())  # Display the first few rows with route and average doors

  New Route Number/Name  AvgDoorsPerRoute
0   Bonnie Doon Stake 1              14.0
1       Gateway Stake 1             144.0
2       Gateway Stake 2             186.0
3       Gateway Stake 3             196.0
4   Bonnie Doon Stake 2               1.0


In [13]:
print(df['Time Spent'].dtype)

float64


In [14]:
average_time_per_door = df['Time Spent'].sum() / df['Doors in Route'].sum()

print(f"Overall average time per door: {average_time_per_door:.2f} minutes")

Overall average time per door: -0.16 minutes


In [15]:
# Group by 'New Route Number/Name' and calculate the total donation volume for each route
route_donation_volume = df.groupby('New Route Number/Name')['Donation Bags Collected'].sum()

# Create a new column 'TotalDonationVolume' and assign the total donation volume for each route using map
df['TotalDonationVolume'] = df['New Route Number/Name'].map(route_donation_volume)

In [16]:
df['Total Volunteers'] = df['# of Youth Volunteers'] + df['# of Adult Volunteers']

In [18]:
df.to_csv('data_2023_donation_processed_2.csv', index=False)

In [19]:
df['New Route Number/Name'].head(50)


,New Route Number/Name
0,Bonnie Doon Stake 1
1,Gateway Stake 1
2,Gateway Stake 2
3,Gateway Stake 3
4,Bonnie Doon Stake 2
5,Gateway Stake 4
6,Gateway Stake 5
7,Bonnie Doon Stake 3
8,Bonnie Doon Stake 4
9,Bonnie Doon Stake 5


In [ ]:
# Check if there are any rows with 'YSA' in the 'Stake' column:
ysa_count = df[df['Stake'] == 'YSA Stake'].shape[0]
print(f"Number of rows with 'YSA' stake: {ysa_count}")

# If ysa_count is 0, then there are no rows with 'YSA' in the 'Stake' column.
if ysa_count == 0:
    print("There are no rows with 'YSA' in the 'Stake' column. "
          "Please check the values in your DataFrame.")
else:
    # Filter the DataFrame to include only rows where the stake is "YSA"
    ysa_df = df[df['Stake'] == 'YSA']

    # Get all route numbers for the YSA stake
    ysa_route_numbers = ysa_df['New Route Number/Name']

    # Print the route numbers
    print(ysa_route_numbers)

Number of rows with 'YSA' stake: 4
Series([], Name: New Route Number/Name, dtype: object)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 12 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Drop Off Location                                      448 non-null    object 
 1   Stake                                                  448 non-null    object 
 2   Route Number/Name                                      448 non-null    object 
 3   Time Spent Collecting Donations                        0 non-null      float64
 4   # of Adult Volunteers who participated in this route   448 non-null    int64  
 5   # of Youth Volunteers who participated in this route
  448 non-null    int64  
 6   # of Doors in Route                                    448 non-null    int64  
 7   # of Donation Bags Collected                           448 non-null    int64  
 8   total number of volunteers                        

In [ ]:
# Group by 'Route Number/Name' and calculate the average '# of Doors in Route' for each group
average_doors_per_route = df.groupby('New Route Number/Name')['# of Doors in Route'].mean()

# Create a new column 'AvgDoorsPerRoute' and assign the average doors for each route using map
df['AvgDoorsPerRoute'] = df['New Route Number/Name'].map(average_doors_per_route)

# Print the updated DataFrame to see the new column
print(df[['New Route Number/Name', 'AvgDoorsPerRoute']].head())  # Display the first few rows with route and average doors

  New Route Number/Name  AvgDoorsPerRoute
0     Riverbend Stake 1              78.0
1       Gateway Stake 1               0.0
2   Bonnie Doon Stake 1               1.0
3       Gateway Stake 2              20.0
4   Bonnie Doon Stake 2              20.0


In [ ]:
df.to_csv('data_2023_donation_processed.csv', index=False)